# 凯利公式 —— 投资组合、风险管理、资金分配

基本公式参考 [凯利公式——单个标的+无风险产品](./kelly_optimal_leverage.ipynb)

## 投资组合公式

C 是标的收益的协方差矩阵，M是标的平均收益列向量。

- $F^* = C^{-1}M$
- $g(F^*) = r + \displaystyle\frac{1}{2} \cdot {{F^*}^T C {F^*}}$
- $sharpe ratio = \sqrt{{F^*}^T C {F^*}}$

# A股主要ETF标的测试

**注意：代码中假定 ETF可以做空，实际A股融券很难，可考虑股指期货。**

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import logging
import log_setup
import helper
import pprint
log_setup.setup()

In [2]:
code2name = helper.datafiles_info()
pprint.pprint(code2name)

{'159601.SZ': 'A50ETF',
 '159781.SZ': '双创50ETF',
 '159915.SZ': '创业板ETF',
 '159949.SZ': '创业板50ETF',
 '510050.SH': '上证50ETF',
 '510300.SH': '沪深300ETF',
 '510500.SH': '中证500ETF',
 '518880.SH': '黄金ETF',
 '588000.SH': '科创50ETF'}


<font color=red>以下计算取无风险利率 $r = 0.02$。</font>

In [3]:
k_annual_risk_free_rate = 0.02 # 假设无风险利率年化为 2%
k_days_per_year = 252

In [4]:
def portfolio_allocation(annual_risk_free=k_annual_risk_free_rate):
    annual_risk_free = k_annual_risk_free_rate
    logging.info('annual_risk_free_rate: {:.4f}'.format(annual_risk_free))

    buffer = []
    for file in [
                 #'./datafiles/159601.SZ_2021-11-08_2023-04-25.xlsx',
                 #'./datafiles/159781.SZ_2021-07-05_2023-04-25.xlsx',
                 './datafiles/159915.SZ_2011-12-09_2023-04-25.xlsx',
                 #'./datafiles/159949.SZ_2016-07-22_2023-04-25.xlsx',
                 #'./datafiles/510050.SH_2005-02-23_2023-04-25.xlsx',
                 './datafiles/510300.SH_2012-05-28_2023-04-25.xlsx',
                 './datafiles/510500.SH_2013-03-15_2023-04-25.xlsx',
                 './datafiles/518880.SH_2013-07-29_2023-04-25.xlsx',
                 #'./datafiles/588000.SH_2020-11-16_2023-04-25.xlsx',
                ]:
        last_slash = file.rfind('/')
        underscore = file.find('_', last_slash)
        code = file[last_slash+1:underscore]
        name = code2name[code]
        df   = pd.read_excel(file, header=0, index_col=0)
        logging.info('{} {} shape {}'.format(code, name, df.shape))
        buffer.append([code, name, df.close])

    data = [item[-1] for item in buffer]
    keys = [item[0] for item in buffer]
    df = pd.concat(data, axis=1, keys=keys)
    logging.info('all data shape: {}'.format(df.shape))

    daily_return         = df.pct_change()
    daily_excess_return  = daily_return - annual_risk_free / k_days_per_year
    annual_excess_return = daily_excess_return.mean(axis=0) * k_days_per_year
    #print(annual_excess_return)

    C = daily_excess_return.dropna(axis=0).cov() * k_days_per_year
    # same as
    # np.cov(daily_excess_return.dropna(axis=0).values, rowvar=False))
    F = np.matmul(np.linalg.inv(C), annual_excess_return.values.reshape(-1,1))
    F = F.reshape(F.shape[0])
    ft_c_f = np.matmul(np.matmul(F.T, C), F)
    g = annual_risk_free + ft_c_f / 2
    sharpe_ratio = ft_c_f
    return F, g, sharpe_ratio

In [5]:
kelly_leverage, comp_ann_growth_rate, sharpe_ratio = portfolio_allocation()
print('kelly leverage: [{}], CAGR: {:.6f}, sharpe ratio: {:.6f}'.format(
    ','.join([ '{:.6f}'.format(l) for l in kelly_leverage ]),
    comp_ann_growth_rate,
    sharpe_ratio))


[2023-04-26 06:56:00,345] [INFO]	[1529384864.py:3:portfolio_allocation] annual_risk_free_rate: 0.0200
[2023-04-26 06:56:01,324] [INFO]	[1529384864.py:22:portfolio_allocation] 159915.SZ 创业板ETF shape (2763, 7)
[2023-04-26 06:56:01,667] [INFO]	[1529384864.py:22:portfolio_allocation] 510300.SH 沪深300ETF shape (2655, 7)
[2023-04-26 06:56:02,023] [INFO]	[1529384864.py:22:portfolio_allocation] 510500.SH 中证500ETF shape (2459, 7)
[2023-04-26 06:56:02,347] [INFO]	[1529384864.py:22:portfolio_allocation] 518880.SH 黄金ETF shape (2372, 7)
[2023-04-26 06:56:02,358] [INFO]	[1529384864.py:28:portfolio_allocation] all data shape: (2764, 4)


kelly leverage: [1.188117,-0.107667,0.001671,2.406027], CAGR: 0.141905, sharpe ratio: 0.243810
